In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-10 02:50:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  5.43MB/s    in 0.2s    

2022-08-10 02:50:19 (5.43 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Team4-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://geisteam4-project.s3.amazonaws.com/ncaqi.csv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("ncaqi.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Show DataFrame
user_data_df.show()

+----+---------+----------+----+-----+---+---+--------+------------------+-------------------------+----------+--------+--------------+-------+--------+----------+-------+----------------+
| _c0|CBSA Code|      Date|year|month|day|AQI|Category|Defining Parameter|Number of Sites Reporting|city_ascii|state_id|    state_name|    lat|     lng|population|density|        timezone|
+----+---------+----------+----+-----+---+---+--------+------------------+-------------------------+----------+--------+--------------+-------+--------+----------+-------+----------------+
|4279|    16740|2022-01-01|2022|    1|  1| 28|    Good|             PM2.5|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303| 1526465.0| 1094.0|America/New_York|
|4280|    16740|2022-01-02|2022|    1|  2| 29|    Good|             Ozone|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303| 1526465.0| 1094.0|America/New_York|
|4281|    16740|2022-01-03|2022|    1|  3| 31|    Good|

In [ ]:
# Print our schema
user_data_df.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: int, CBSA Code: int, Date: string, year: int, month: int, day: int, AQI: int, Category: string, Defining Parameter: string, Number of Sites Reporting: int, city_ascii: string, state_id: string, state_name: string, lat: double, lng: double, population: double, density: double, timezone: string]>

In [ ]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType, DecimalType, FloatType

In [ ]:
# Create the list of struct fields
schema = [StructField("_c0",IntegerType(),False),StructField("CBSA Code", IntegerType(),False), StructField("Date", DateType(),False), StructField("year",IntegerType(), False), StructField("month",StringType(), False),StructField("day",IntegerType(),False), StructField("AQI", IntegerType(), False), StructField("Category",StringType(), False), StructField("Defining Parameter",StringType(),False), StructField("Number of Sites Reporting",IntegerType(),False), StructField("city_ascii",StringType(),False), StructField("state_id",StringType(),False),StructField("state_name",StringType(),False), StructField("lat",FloatType(),False),StructField("lng",FloatType(),False),StructField("population",FloatType(),False),StructField("density",FloatType(),False), StructField("timezone",StringType(),False)]
schema

[StructField(_c0,IntegerType,false),
 StructField(CBSA Code,IntegerType,false),
 StructField(Date,DateType,false),
 StructField(year,IntegerType,false),
 StructField(month,StringType,false),
 StructField(day,IntegerType,false),
 StructField(AQI,IntegerType,false),
 StructField(Category,StringType,false),
 StructField(Defining Parameter,StringType,false),
 StructField(Number of Sites Reporting,IntegerType,false),
 StructField(city_ascii,StringType,false),
 StructField(state_id,StringType,false),
 StructField(state_name,StringType,false),
 StructField(lat,FloatType,false),
 StructField(lng,FloatType,false),
 StructField(population,FloatType,false),
 StructField(density,FloatType,false),
 StructField(timezone,StringType,false)]

In [ ]:
# Pass in our fields
final = StructType(fields=schema)
final

StructType(List(StructField(_c0,IntegerType,false),StructField(CBSA Code,IntegerType,false),StructField(Date,DateType,false),StructField(year,IntegerType,false),StructField(month,StringType,false),StructField(day,IntegerType,false),StructField(AQI,IntegerType,false),StructField(Category,StringType,false),StructField(Defining Parameter,StringType,false),StructField(Number of Sites Reporting,IntegerType,false),StructField(city_ascii,StringType,false),StructField(state_id,StringType,false),StructField(state_name,StringType,false),StructField(lat,FloatType,false),StructField(lng,FloatType,false),StructField(population,FloatType,false),StructField(density,FloatType,false),StructField(timezone,StringType,false)))

In [ ]:
# Update dataframe based on new schema
user_data_df1 = spark.read.csv(SparkFiles.get("ncaqi.csv"),schema=final, sep=",", header=True)
user_data_df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- CBSA Code: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- AQI: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Defining Parameter: string (nullable = true)
 |-- Number of Sites Reporting: integer (nullable = true)
 |-- city_ascii: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lng: float (nullable = true)
 |-- population: float (nullable = true)
 |-- density: float (nullable = true)
 |-- timezone: string (nullable = true)



In [ ]:
# Create dataframe to match AirQuality table
airquality_df = user_data_df1.select(['_c0','CBSA Code','Date','year','month','day','AQI','Category','Defining Parameter'])
airquality_df.show()

+----+---------+----------+----+-----+---+---+--------+------------------+
| _c0|CBSA Code|      Date|year|month|day|AQI|Category|Defining Parameter|
+----+---------+----------+----+-----+---+---+--------+------------------+
|4279|    16740|2022-01-01|2022|    1|  1| 28|    Good|             PM2.5|
|4280|    16740|2022-01-02|2022|    1|  2| 29|    Good|             Ozone|
|4281|    16740|2022-01-03|2022|    1|  3| 31|    Good|             Ozone|
|4282|    16740|2022-01-04|2022|    1|  4| 31|    Good|             Ozone|
|4283|    16740|2022-01-05|2022|    1|  5| 34|    Good|             Ozone|
|4284|    16740|2022-01-06|2022|    1|  6| 42|    Good|             PM2.5|
|4285|    16740|2022-01-07|2022|    1|  7| 32|    Good|             Ozone|
|4286|    16740|2022-01-08|2022|    1|  8| 33|    Good|             PM2.5|
|4287|    16740|2022-01-09|2022|    1|  9| 38|    Good|             PM2.5|
|4288|    16740|2022-01-10|2022|    1| 10| 36|    Good|             Ozone|
|4289|    16740|2022-01-1

In [ ]:
# Create the dataframe to match Site Reporting Table
site_reporting_df = user_data_df1.select(['_c0','CBSA Code', 'Number of Sites Reporting', 'city_ascii','state_id', 'state_name', 'lat','lng'])
site_reporting_df.show()

+----+---------+-------------------------+----------+--------+--------------+-------+--------+
| _c0|CBSA Code|Number of Sites Reporting|city_ascii|state_id|    state_name|    lat|     lng|
+----+---------+-------------------------+----------+--------+--------------+-------+--------+
|4279|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4280|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4281|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4282|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4283|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4284|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4285|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4286|    16740|                        5| Charlot

In [ ]:
# Rename column names
airquality_df1= airquality_df.withColumnRenamed("CBSA Code","CBSA_Code")\
               .withColumnRenamed("Defining Parameter","Defining_Parameter")
airquality_df1.show()

+----+---------+----------+----+-----+---+---+--------+------------------+
| _c0|CBSA_Code|      Date|year|month|day|AQI|Category|Defining_Parameter|
+----+---------+----------+----+-----+---+---+--------+------------------+
|4279|    16740|2022-01-01|2022|    1|  1| 28|    Good|             PM2.5|
|4280|    16740|2022-01-02|2022|    1|  2| 29|    Good|             Ozone|
|4281|    16740|2022-01-03|2022|    1|  3| 31|    Good|             Ozone|
|4282|    16740|2022-01-04|2022|    1|  4| 31|    Good|             Ozone|
|4283|    16740|2022-01-05|2022|    1|  5| 34|    Good|             Ozone|
|4284|    16740|2022-01-06|2022|    1|  6| 42|    Good|             PM2.5|
|4285|    16740|2022-01-07|2022|    1|  7| 32|    Good|             Ozone|
|4286|    16740|2022-01-08|2022|    1|  8| 33|    Good|             PM2.5|
|4287|    16740|2022-01-09|2022|    1|  9| 38|    Good|             PM2.5|
|4288|    16740|2022-01-10|2022|    1| 10| 36|    Good|             Ozone|
|4289|    16740|2022-01-1

In [ ]:
airquality_df1.dtypes


[('_c0', 'int'),
 ('CBSA_Code', 'int'),
 ('Date', 'date'),
 ('year', 'int'),
 ('month', 'string'),
 ('day', 'int'),
 ('AQI', 'int'),
 ('Category', 'string'),
 ('Defining_Parameter', 'string')]

In [ ]:
#Rename column names
site_reporting_df1= site_reporting_df.withColumnRenamed("CBSA Code","CBSA_Code")\
               .withColumnRenamed("Number of Sites Reporting","Number_of_Sites_Reporting")
site_reporting_df1.show()

+----+---------+-------------------------+----------+--------+--------------+-------+--------+
| _c0|CBSA_Code|Number_of_Sites_Reporting|city_ascii|state_id|    state_name|    lat|     lng|
+----+---------+-------------------------+----------+--------+--------------+-------+--------+
|4279|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4280|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4281|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4282|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4283|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4284|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4285|    16740|                        5| Charlotte|      NC|North Carolina|35.2083|-80.8303|
|4286|    16740|                        5| Charlot

In [ ]:
site_reporting_df1.dtypes


[('_c0', 'int'),
 ('CBSA_Code', 'int'),
 ('Number_of_Sites_Reporting', 'int'),
 ('city_ascii', 'string'),
 ('state_id', 'string'),
 ('state_name', 'string'),
 ('lat', 'float'),
 ('lng', 'float')]

In [ ]:
# Create the dataframe to match Population table
population_df = user_data_df1.select(['_c0','CBSA Code','city_ascii','state_id', 'population', 'density','timezone'])
population_df.show()

+----+---------+----------+--------+----------+-------+----------------+
| _c0|CBSA Code|city_ascii|state_id|population|density|        timezone|
+----+---------+----------+--------+----------+-------+----------------+
|4279|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4280|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4281|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4282|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4283|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4284|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4285|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4286|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4287|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4288|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4289|    16740| Charlotte|      NC| 1526465.0| 109

In [ ]:
#Rename Column Names
population_df1= population_df.withColumnRenamed("CBSA Code","CBSA_Code")
population_df1.show()

+----+---------+----------+--------+----------+-------+----------------+
| _c0|CBSA_Code|city_ascii|state_id|population|density|        timezone|
+----+---------+----------+--------+----------+-------+----------------+
|4279|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4280|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4281|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4282|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4283|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4284|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4285|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4286|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4287|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4288|    16740| Charlotte|      NC| 1526465.0| 1094.0|America/New_York|
|4289|    16740| Charlotte|      NC| 1526465.0| 109

In [ ]:
population_df1.dtypes

[('_c0', 'int'),
 ('CBSA_Code', 'int'),
 ('city_ascii', 'string'),
 ('state_id', 'string'),
 ('population', 'float'),
 ('density', 'float'),
 ('timezone', 'string')]

In [ ]:
from pyspark.sql.functions import to_date


In [ ]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://geisteam4.coe2ggfhl77s.us-east-1.rds.amazonaws.com"
jdbc_url="jdbc:postgresql://geisteam4.coe2ggfhl77s.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "geisadmin01", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write airquality_df to table in RDS
airquality_df1.write.jdbc(url=jdbc_url, table='air_quality_table', mode=mode, properties=config)

In [ ]:
# Write site_reporting_df to table in RDS
site_reporting_df1.write.jdbc(url=jdbc_url, table='site_reporting_table', mode=mode, properties=config)

In [ ]:
# Write site_reporting_df to table in RDS
population_df1.write.jdbc(url=jdbc_url, table='population_table', mode=mode, properties=config)